In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "accelerator": "GPU",
    "colab": {
      "name": "NumberGAN_Solution.ipynb",
      "provenance": [],
      "collapsed_sections": []
    },
    "kernelspec": {
      "display_name": "Python 3",
      "name": "python3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "NJp-D51g0IDd"
      },
      "source": [
        "## **1) Importing Python Packages for GAN**\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "1k5mFBuzzl2a"
      },
      "source": [
        "from keras.datasets import mnist\n",
        "\n",
        "from keras.models import Sequential\n",
        "from keras.layers import BatchNormalization\n",
        "from keras.layers import Dense, Reshape, Flatten\n",
        "from keras.layers.advanced_activations import LeakyReLU\n",
        "from tensorflow.keras.optimizers import Adam\n",
        "\n",
        "import numpy as np\n",
        "!mkdir generated_images"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Yr-eZOzg0X79"
      },
      "source": [
        "## **2) Variables for Neural Networks & Data**"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "RThZMDruz9cB",
        "outputId": "ac040e71-9a75-4061-ef7d-5361f31bc976"
      },
      "source": [
        "img_width = 28\n",
        "img_height = 28\n",
        "channels = 1\n",
        "img_shape = (img_width, img_height, channels)\n",
        "latent_dim = 100\n",
        "adam = Adam(lr=0.0001)"
      ],
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stderr",
          "text": [
            "/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.\n",
            "  super(Adam, self).__init__(name, **kwargs)\n"
          ]
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "U3bcJZZg0cqy"
      },
      "source": [
        "## **3) Building Generator**\n",
        "\n",
        "\n",
        "\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "NdiqZpri0iQh",
        "outputId": "166dfc32-17f5-4613-d90a-d523b538d76f",
        "colab": {
          "base_uri": "https://localhost:8080/"
        }
      },
      "source": [
        "def build_generator():\n",
        "  model = Sequential()\n",
        "\n",
        "  model.add(Dense(256, input_dim=latent_dim))\n",
        "  model.add(LeakyReLU(alpha=0.2))\n",
        "  model.add(BatchNormalization(momentum=0.8))\n",
        "\n",
        "  model.add(Dense(256))\n",
        "  model.add(LeakyReLU(alpha=0.2))\n",
        "  model.add(BatchNormalization(momentum=0.8))\n",
        "\n",
        "  model.add(Dense(256))\n",
        "  model.add(LeakyReLU(alpha=0.2))\n",
        "  model.add(BatchNormalization(momentum=0.8))\n",
        "\n",
        "  model.add(Dense(np.prod(img_shape), activation='tanh'))\n",
        "  model.add(Reshape(img_shape))\n",
        "  \n",
        "  model.summary()\n",
        "  return model\n",
        "\n",
        "generator = build_generator()"
      ],
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Model: \"sequential\"\n",
            "_________________________________________________________________\n",
            " Layer (type)                Output Shape              Param #   \n",
            "=================================================================\n",
            " dense (Dense)               (None, 256)               25856     \n",
            "                                                                 \n",
            " leaky_re_lu (LeakyReLU)     (None, 256)               0         \n",
            "                                                                 \n",
            " batch_normalization (BatchN  (None, 256)              1024      \n",
            " ormalization)                                                   \n",
            "                                                                 \n",
            " dense_1 (Dense)             (None, 256)               65792     \n",
            "                                                                 \n",
            " leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         \n",
            "                                                                 \n",
            " batch_normalization_1 (Batc  (None, 256)              1024      \n",
            " hNormalization)                                                 \n",
            "                                                                 \n",
            " dense_2 (Dense)             (None, 256)               65792     \n",
            "                                                                 \n",
            " leaky_re_lu_2 (LeakyReLU)   (None, 256)               0         \n",
            "                                                                 \n",
            " batch_normalization_2 (Batc  (None, 256)              1024      \n",
            " hNormalization)                                                 \n",
            "                                                                 \n",
            " dense_3 (Dense)             (None, 784)               201488    \n",
            "                                                                 \n",
            " reshape (Reshape)           (None, 28, 28, 1)         0         \n",
            "                                                                 \n",
            "=================================================================\n",
            "Total params: 362,000\n",
            "Trainable params: 360,464\n",
            "Non-trainable params: 1,536\n",
            "_________________________________________________________________\n"
          ]
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Bt6QsJCW0mcI"
      },
      "source": [
        "## **4) Building Discriminator**"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "V2JzEAPv0lKt",
        "outputId": "c5dc2705-5b5f-4c83-89f5-731ae33203ec"
      },
      "source": [
        "def build_discriminator():\n",
        "  model = Sequential()\n",
        "\n",
        "  model.add(Flatten(input_shape=img_shape))\n",
        "  model.add(Dense(512))\n",
        "  model.add(LeakyReLU(alpha=0.2))\n",
        "  model.add(Dense(256))\n",
        "  model.add(Dense(1, activation='sigmoid'))\n",
        "\n",
        "  model.summary()\n",
        "  return model\n",
        "\n",
        "discriminator = build_discriminator()\n",
        "discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])"
      ],
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Model: \"sequential_1\"\n",
            "_________________________________________________________________\n",
            " Layer (type)                Output Shape              Param #   \n",
            "=================================================================\n",
            " flatten (Flatten)           (None, 784)               0         \n",
            "                                                                 \n",
            " dense_4 (Dense)             (None, 512)               401920    \n",
            "                                                                 \n",
            " leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         \n",
            "                                                                 \n",
            " dense_5 (Dense)             (None, 256)               131328    \n",
            "                                                                 \n",
            " dense_6 (Dense)             (None, 1)                 257       \n",
            "                                                                 \n",
            "=================================================================\n",
            "Total params: 533,505\n",
            "Trainable params: 533,505\n",
            "Non-trainable params: 0\n",
            "_________________________________________________________________\n"
          ]
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "TbcKcKmA0q2S"
      },
      "source": [
        "## **5) Connecting Neural Networks to build GAN**"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "q0Ue3TEd0xLy",
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "outputId": "491bfb8d-db71-4ef5-c636-8a3c918b64d5"
      },
      "source": [
        "GAN = Sequential()\n",
        "discriminator.trainable = False\n",
        "GAN.add(generator)\n",
        "GAN.add(discriminator)\n",
        "\n",
        "GAN.compile(loss='binary_crossentropy', optimizer=adam)\n",
        "GAN.summary()\n"
      ],
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Model: \"sequential_2\"\n",
            "_________________________________________________________________\n",
            " Layer (type)                Output Shape              Param #   \n",
            "=================================================================\n",
            " sequential (Sequential)     (None, 28, 28, 1)         362000    \n",
            "                                                                 \n",
            " sequential_1 (Sequential)   (None, 1)                 533505    \n",
            "                                                                 \n",
            "=================================================================\n",
            "Total params: 895,505\n",
            "Trainable params: 360,464\n",
            "Non-trainable params: 535,041\n",
            "_________________________________________________________________\n"
          ]
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "2WaNhBDwRwTG"
      },
      "source": [
        "## **6) Outputting Images**\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "HQEJ0WbjRppy"
      },
      "source": [
        "#@title\n",
        "## **7) Outputting Images**\n",
        "import matplotlib.pyplot as plt\n",
        "import glob\n",
        "import imageio\n",
        "import PIL\n",
        "\n",
        "save_name = 0.00000000\n",
        "\n",
        "def save_imgs(epoch):\n",
        "    r, c = 5, 5\n",
        "    noise = np.random.normal(0, 1, (r * c, latent_dim))\n",
        "    gen_imgs = generator.predict(noise)\n",
        "    global save_name\n",
        "    save_name += 0.00000001\n",
        "    print(\"%.8f\" % save_name)\n",
        "\n",
        "    # Rescale images 0 - 1\n",
        "    gen_imgs = 0.5 * gen_imgs + 0.5\n",
        "\n",
        "    fig, axs = plt.subplots(r, c)\n",
        "    cnt = 0\n",
        "    for i in range(r):\n",
        "        for j in range(c):\n",
        "            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')\n",
        "            # axs[i,j].imshow(gen_imgs[cnt])\n",
        "            axs[i,j].axis('off')\n",
        "            cnt += 1\n",
        "    fig.savefig(\"generated_images/%.8f.png\" % save_name)\n",
        "    print('saved')\n",
        "    plt.close()"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "tE57Lk5V0xs2"
      },
      "source": [
        "## **7) Training GAN**"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "egSJJvik00Iq",
        "outputId": "31c8c38f-dafd-41b3-e976-4f68473a9922",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 1000
        }
      },
      "source": [
        "def train(epochs, batch_size=64, save_interval=200):\n",
        "  (X_train, _), (_, _) = mnist.load_data()\n",
        "\n",
        "  # print(X_train.shape)\n",
        "  #Rescale data between -1 and 1\n",
        "  X_train = X_train / 127.5 -1.\n",
        "  # X_train = np.expand_dims(X_train, axis=3)\n",
        "  # print(X_train.shape)\n",
        "\n",
        "  #Create our Y for our Neural Networks\n",
        "  valid = np.ones((batch_size, 1))\n",
        "  fakes = np.zeros((batch_size, 1))\n",
        "\n",
        "  for epoch in range(epochs):\n",
        "    #Get Random Batch\n",
        "    idx = np.random.randint(0, X_train.shape[0], batch_size)\n",
        "    imgs = X_train[idx]\n",
        "\n",
        "    #Generate Fake Images\n",
        "    noise = np.random.normal(0, 1, (batch_size, latent_dim))\n",
        "    gen_imgs = generator.predict(noise)\n",
        "\n",
        "    #Train discriminator\n",
        "    d_loss_real = discriminator.train_on_batch(imgs, valid)\n",
        "    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)\n",
        "    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)\n",
        "\n",
        "    noise = np.random.normal(0, 1, (batch_size, latent_dim))\n",
        "    \n",
        "    #inverse y label\n",
        "    g_loss = GAN.train_on_batch(noise, valid)\n",
        "\n",
        "    print(\"******* %d [D loss: %f, acc: %.2f%%] [G loss: %f]\" % (epoch, d_loss[0], 100* d_loss[1], g_loss))\n",
        "\n",
        "    if(epoch % save_interval) == 0:\n",
        "      save_imgs(epoch)\n",
        "\n",
        "  # print(valid)\n",
        "\n",
        "\n",
        "train(30000, batch_size=64, save_interval=200)"
      ],
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz\n",
            "11493376/11490434 [==============================] - 0s 0us/step\n",
            "11501568/11490434 [==============================] - 0s 0us/step\n",
            "******* 0 [D loss: 0.887595, acc: 45.31%] [G loss: 0.767116]\n",
            "0.00000001\n",
            "saved\n",
            "******* 1 [D loss: 0.405668, acc: 79.69%] [G loss: 0.748326]\n",
            "******* 2 [D loss: 0.345956, acc: 80.47%] [G loss: 0.739114]\n",
            "******* 3 [D loss: 0.333020, acc: 82.81%] [G loss: 0.706251]\n",
            "******* 4 [D loss: 0.342352, acc: 78.12%] [G loss: 0.756652]\n",
            "******* 5 [D loss: 0.337693, acc: 79.69%] [G loss: 0.819206]\n",
            "******* 6 [D loss: 0.320905, acc: 78.91%] [G loss: 0.855542]\n",
            "******* 7 [D loss: 0.306857, acc: 82.81%] [G loss: 0.936298]\n",
            "******* 8 [D loss: 0.293247, acc: 89.84%] [G loss: 1.036027]\n",
            "******* 9 [D loss: 0.255740, acc: 94.53%] [G loss: 1.078977]\n",
            "******* 10 [D loss: 0.257833, acc: 90.62%] [G loss: 1.192591]\n",
            "******* 11 [D loss: 0.220323, acc: 96.88%] [G loss: 1.303853]\n",
            "******* 12 [D loss: 0.190162, acc: 100.00%] [G loss: 1.428722]\n",
            "******* 13 [D loss: 0.154554, acc: 99.22%] [G loss: 1.608374]\n",
            "******* 14 [D loss: 0.127561, acc: 100.00%] [G loss: 1.714983]\n",
            "******* 15 [D loss: 0.111209, acc: 100.00%] [G loss: 1.838400]\n",
            "******* 16 [D loss: 0.099721, acc: 100.00%] [G loss: 1.952587]\n",
            "******* 17 [D loss: 0.087172, acc: 100.00%] [G loss: 2.059102]\n",
            "******* 18 [D loss: 0.071507, acc: 100.00%] [G loss: 2.179142]\n",
            "******* 19 [D loss: 0.070814, acc: 100.00%] [G loss: 2.304069]\n",
            "******* 20 [D loss: 0.057023, acc: 100.00%] [G loss: 2.445385]\n",
            "******* 21 [D loss: 0.053082, acc: 100.00%] [G loss: 2.542961]\n",
            "******* 22 [D loss: 0.051527, acc: 100.00%] [G loss: 2.610686]\n",
            "******* 23 [D loss: 0.044485, acc: 100.00%] [G loss: 2.631511]\n",
            "******* 24 [D loss: 0.038197, acc: 100.00%] [G loss: 2.727536]\n",
            "******* 25 [D loss: 0.038307, acc: 100.00%] [G loss: 2.784837]\n",
            "******* 26 [D loss: 0.039958, acc: 100.00%] [G loss: 2.876319]\n",
            "******* 27 [D loss: 0.035662, acc: 100.00%] [G loss: 2.928057]\n",
            "******* 28 [D loss: 0.031384, acc: 100.00%] [G loss: 2.962376]\n",
            "******* 29 [D loss: 0.029348, acc: 100.00%] [G loss: 3.014545]\n",
            "******* 30 [D loss: 0.030065, acc: 100.00%] [G loss: 3.075484]\n",
            "******* 31 [D loss: 0.026830, acc: 100.00%] [G loss: 3.062098]\n",
            "******* 32 [D loss: 0.027748, acc: 100.00%] [G loss: 3.062397]\n",
            "******* 33 [D loss: 0.025648, acc: 100.00%] [G loss: 3.101132]\n",
            "******* 34 [D loss: 0.028176, acc: 100.00%] [G loss: 3.107950]\n",
            "******* 35 [D loss: 0.025816, acc: 100.00%] [G loss: 3.163767]\n",
            "******* 36 [D loss: 0.026522, acc: 100.00%] [G loss: 3.240295]\n",
            "******* 37 [D loss: 0.026216, acc: 100.00%] [G loss: 3.189269]\n",
            "******* 38 [D loss: 0.024015, acc: 100.00%] [G loss: 3.169399]\n",
            "******* 39 [D loss: 0.022231, acc: 100.00%] [G loss: 3.115056]\n",
            "******* 40 [D loss: 0.027552, acc: 100.00%] [G loss: 3.181084]\n",
            "******* 41 [D loss: 0.022812, acc: 100.00%] [G loss: 3.193994]\n",
            "******* 42 [D loss: 0.024095, acc: 100.00%] [G loss: 3.172971]\n",
            "******* 43 [D loss: 0.022707, acc: 100.00%] [G loss: 3.139848]\n",
            "******* 44 [D loss: 0.021686, acc: 100.00%] [G loss: 3.289233]\n",
            "******* 45 [D loss: 0.020121, acc: 100.00%] [G loss: 3.285497]\n",
            "******* 46 [D loss: 0.021227, acc: 100.00%] [G loss: 3.194090]\n",
            "******* 47 [D loss: 0.021577, acc: 100.00%] [G loss: 3.272365]\n",
            "******* 48 [D loss: 0.020971, acc: 100.00%] [G loss: 3.294955]\n",
            "******* 49 [D loss: 0.023383, acc: 100.00%] [G loss: 3.324584]\n",
            "******* 50 [D loss: 0.024454, acc: 100.00%] [G loss: 3.295917]\n",
            "******* 51 [D loss: 0.020467, acc: 100.00%] [G loss: 3.298032]\n",
            "******* 52 [D loss: 0.020744, acc: 100.00%] [G loss: 3.281011]\n",
            "******* 53 [D loss: 0.022824, acc: 100.00%] [G loss: 3.300265]\n",
            "******* 54 [D loss: 0.024983, acc: 100.00%] [G loss: 3.380123]\n",
            "******* 55 [D loss: 0.021341, acc: 100.00%] [G loss: 3.279719]\n",
            "******* 56 [D loss: 0.022923, acc: 100.00%] [G loss: 3.344484]\n",
            "******* 57 [D loss: 0.021996, acc: 100.00%] [G loss: 3.402670]\n",
            "******* 58 [D loss: 0.020633, acc: 100.00%] [G loss: 3.335642]\n",
            "******* 59 [D loss: 0.020644, acc: 100.00%] [G loss: 3.330906]\n",
            "******* 60 [D loss: 0.020034, acc: 100.00%] [G loss: 3.470451]\n",
            "******* 61 [D loss: 0.020317, acc: 100.00%] [G loss: 3.445269]\n",
            "******* 62 [D loss: 0.017723, acc: 100.00%] [G loss: 3.494688]\n",
            "******* 63 [D loss: 0.017043, acc: 100.00%] [G loss: 3.413889]\n",
            "******* 64 [D loss: 0.018582, acc: 100.00%] [G loss: 3.403938]\n",
            "******* 65 [D loss: 0.017904, acc: 100.00%] [G loss: 3.410262]\n",
            "******* 66 [D loss: 0.020047, acc: 100.00%] [G loss: 3.455626]\n",
            "******* 67 [D loss: 0.021032, acc: 100.00%] [G loss: 3.537026]\n",
            "******* 68 [D loss: 0.016465, acc: 100.00%] [G loss: 3.526901]\n",
            "******* 69 [D loss: 0.019209, acc: 100.00%] [G loss: 3.425596]\n",
            "******* 70 [D loss: 0.018885, acc: 100.00%] [G loss: 3.442147]\n",
            "******* 71 [D loss: 0.016517, acc: 100.00%] [G loss: 3.517414]\n",
            "******* 72 [D loss: 0.018187, acc: 100.00%] [G loss: 3.487462]\n",
            "******* 73 [D loss: 0.018853, acc: 100.00%] [G loss: 3.623919]\n",
            "******* 74 [D loss: 0.019464, acc: 100.00%] [G loss: 3.580263]\n",
            "******* 75 [D loss: 0.017221, acc: 100.00%] [G loss: 3.565736]\n",
            "******* 76 [D loss: 0.017446, acc: 100.00%] [G loss: 3.607656]\n",
            "******* 77 [D loss: 0.016478, acc: 100.00%] [G loss: 3.635428]\n",
            "******* 78 [D loss: 0.014950, acc: 100.00%] [G loss: 3.642250]\n",
            "******* 79 [D loss: 0.016156, acc: 100.00%] [G loss: 3.589338]\n",
            "******* 80 [D loss: 0.018331, acc: 100.00%] [G loss: 3.656942]\n",
            "******* 81 [D loss: 0.014726, acc: 100.00%] [G loss: 3.678287]\n",
            "******* 82 [D loss: 0.015382, acc: 100.00%] [G loss: 3.660866]\n",
            "******* 83 [D loss: 0.015488, acc: 100.00%] [G loss: 3.605052]\n",
            "******* 84 [D loss: 0.013401, acc: 100.00%] [G loss: 3.606966]\n",
            "******* 85 [D loss: 0.016101, acc: 100.00%] [G loss: 3.627815]\n",
            "******* 86 [D loss: 0.014191, acc: 100.00%] [G loss: 3.718097]\n",
            "******* 87 [D loss: 0.014471, acc: 100.00%] [G loss: 3.624797]\n",
            "******* 88 [D loss: 0.014437, acc: 100.00%] [G loss: 3.716690]\n",
            "******* 89 [D loss: 0.016792, acc: 100.00%] [G loss: 3.671373]\n",
            "******* 90 [D loss: 0.013195, acc: 100.00%] [G loss: 3.625812]\n",
            "******* 91 [D loss: 0.014900, acc: 100.00%] [G loss: 3.671586]\n",
            "******* 92 [D loss: 0.014911, acc: 100.00%] [G loss: 3.645650]\n",
            "******* 93 [D loss: 0.016027, acc: 100.00%] [G loss: 3.782292]\n",
            "******* 94 [D loss: 0.012193, acc: 100.00%] [G loss: 3.710413]\n",
            "******* 95 [D loss: 0.014391, acc: 100.00%] [G loss: 3.665170]\n",
            "******* 96 [D loss: 0.014872, acc: 100.00%] [G loss: 3.713327]\n",
            "******* 97 [D loss: 0.014552, acc: 100.00%] [G loss: 3.784928]\n",
            "******* 98 [D loss: 0.014155, acc: 100.00%] [G loss: 3.741305]\n",
            "******* 99 [D loss: 0.015092, acc: 100.00%] [G loss: 3.782977]\n",
            "******* 100 [D loss: 0.011584, acc: 100.00%] [G loss: 3.801511]\n",
            "******* 101 [D loss: 0.012497, acc: 100.00%] [G loss: 3.818971]\n",
            "******* 102 [D loss: 0.014930, acc: 100.00%] [G loss: 3.809093]\n",
            "******* 103 [D loss: 0.014322, acc: 100.00%] [G loss: 3.833256]\n",
            "******* 104 [D loss: 0.013602, acc: 100.00%] [G loss: 3.793944]\n",
            "******* 105 [D loss: 0.014410, acc: 100.00%] [G loss: 3.803089]\n",
            "******* 106 [D loss: 0.013783, acc: 100.00%] [G loss: 3.939082]\n",
            "******* 107 [D loss: 0.015252, acc: 100.00%] [G loss: 3.883614]\n",
            "******* 108 [D loss: 0.015102, acc: 100.00%] [G loss: 3.796621]\n",
            "******* 109 [D loss: 0.013292, acc: 100.00%] [G loss: 3.895370]\n",
            "******* 110 [D loss: 0.012461, acc: 100.00%] [G loss: 3.905505]\n",
            "******* 111 [D loss: 0.014754, acc: 100.00%] [G loss: 3.892162]\n",
            "******* 112 [D loss: 0.013246, acc: 100.00%] [G loss: 3.945469]\n",
            "******* 113 [D loss: 0.012612, acc: 100.00%] [G loss: 3.985584]\n",
            "******* 114 [D loss: 0.010890, acc: 100.00%] [G loss: 3.948858]\n",
            "******* 115 [D loss: 0.011326, acc: 100.00%] [G loss: 3.890499]\n",
            "******* 116 [D loss: 0.012697, acc: 100.00%] [G loss: 4.002373]\n",
            "******* 117 [D loss: 0.012974, acc: 100.00%] [G loss: 3.930768]\n",
            "******* 118 [D loss: 0.011039, acc: 100.00%] [G loss: 3.949944]\n",
            "******* 119 [D loss: 0.011074, acc: 100.00%] [G loss: 3.998259]\n",
            "******* 120 [D loss: 0.011042, acc: 100.00%] [G loss: 3.986333]\n",
            "******* 121 [D loss: 0.011805, acc: 100.00%] [G loss: 3.995777]\n",
            "******* 122 [D loss: 0.013568, acc: 100.00%] [G loss: 3.985265]\n",
            "******* 123 [D loss: 0.012082, acc: 100.00%] [G loss: 3.960634]\n",
            "******* 124 [D loss: 0.011091, acc: 100.00%] [G loss: 3.933666]\n",
            "******* 125 [D loss: 0.010086, acc: 100.00%] [G loss: 3.980640]\n",
            "******* 126 [D loss: 0.013468, acc: 100.00%] [G loss: 3.985919]\n",
            "******* 127 [D loss: 0.010646, acc: 100.00%] [G loss: 4.002507]\n",
            "******* 128 [D loss: 0.011896, acc: 100.00%] [G loss: 4.094346]\n",
            "******* 129 [D loss: 0.011810, acc: 100.00%] [G loss: 4.002273]\n",
            "******* 130 [D loss: 0.012047, acc: 100.00%] [G loss: 3.963717]\n",
            "******* 131 [D loss: 0.010916, acc: 100.00%] [G loss: 4.052400]\n",
            "******* 132 [D loss: 0.011625, acc: 100.00%] [G loss: 4.096560]\n",
            "******* 133 [D loss: 0.012737, acc: 100.00%] [G loss: 3.988784]\n",
            "******* 134 [D loss: 0.013214, acc: 100.00%] [G loss: 4.032121]\n",
            "******* 135 [D loss: 0.011826, acc: 100.00%] [G loss: 4.120291]\n",
            "******* 136 [D loss: 0.012068, acc: 100.00%] [G loss: 4.089785]\n",
            "******* 137 [D loss: 0.012541, acc: 100.00%] [G loss: 4.083216]\n",
            "******* 138 [D loss: 0.011955, acc: 100.00%] [G loss: 4.098425]\n",
            "******* 139 [D loss: 0.012130, acc: 100.00%] [G loss: 4.087247]\n",
            "******* 140 [D loss: 0.010949, acc: 100.00%] [G loss: 4.084918]\n",
            "******* 141 [D loss: 0.013981, acc: 100.00%] [G loss: 4.190564]\n",
            "******* 142 [D loss: 0.010937, acc: 100.00%] [G loss: 4.140103]\n",
            "******* 143 [D loss: 0.010748, acc: 100.00%] [G loss: 4.162172]\n",
            "******* 144 [D loss: 0.009542, acc: 100.00%] [G loss: 4.200652]\n",
            "******* 145 [D loss: 0.010746, acc: 100.00%] [G loss: 4.171209]\n",
            "******* 146 [D loss: 0.010589, acc: 100.00%] [G loss: 4.136430]\n",
            "******* 147 [D loss: 0.010427, acc: 100.00%] [G loss: 4.103925]\n",
            "******* 148 [D loss: 0.011179, acc: 100.00%] [G loss: 4.218429]\n",
            "******* 149 [D loss: 0.009774, acc: 100.00%] [G loss: 4.117167]\n",
            "******* 150 [D loss: 0.007518, acc: 100.00%] [G loss: 4.169094]\n",
            "******* 151 [D loss: 0.011142, acc: 100.00%] [G loss: 4.203152]\n",
            "******* 152 [D loss: 0.012431, acc: 100.00%] [G loss: 4.154524]\n",
            "******* 153 [D loss: 0.011954, acc: 100.00%] [G loss: 4.204410]\n",
            "******* 154 [D loss: 0.012326, acc: 100.00%] [G loss: 4.136154]\n",
            "******* 155 [D loss: 0.011123, acc: 100.00%] [G loss: 4.180346]\n",
            "******* 156 [D loss: 0.013107, acc: 100.00%] [G loss: 4.183456]\n",
            "******* 157 [D loss: 0.012936, acc: 100.00%] [G loss: 4.209165]\n",
            "******* 158 [D loss: 0.009622, acc: 100.00%] [G loss: 4.163790]\n",
            "******* 159 [D loss: 0.010726, acc: 100.00%] [G loss: 4.167893]\n",
            "******* 160 [D loss: 0.010121, acc: 100.00%] [G loss: 4.178885]\n",
            "******* 161 [D loss: 0.010749, acc: 100.00%] [G loss: 4.238237]\n",
            "******* 162 [D loss: 0.012310, acc: 100.00%] [G loss: 4.186337]\n",
            "******* 163 [D loss: 0.010503, acc: 100.00%] [G loss: 4.124057]\n",
            "******* 164 [D loss: 0.010563, acc: 100.00%] [G loss: 4.205729]\n",
            "******* 165 [D loss: 0.011089, acc: 100.00%] [G loss: 4.197129]\n",
            "******* 166 [D loss: 0.010497, acc: 100.00%] [G loss: 4.156010]\n",
            "******* 167 [D loss: 0.011136, acc: 100.00%] [G loss: 4.236703]\n",
            "******* 168 [D loss: 0.010258, acc: 100.00%] [G loss: 4.246906]\n",
            "******* 169 [D loss: 0.009465, acc: 100.00%] [G loss: 4.223343]\n",
            "******* 170 [D loss: 0.010778, acc: 100.00%] [G loss: 4.217636]\n",
            "******* 171 [D loss: 0.008611, acc: 100.00%] [G loss: 4.254766]\n",
            "******* 172 [D loss: 0.011502, acc: 100.00%] [G loss: 4.221816]\n",
            "******* 173 [D loss: 0.010251, acc: 100.00%] [G loss: 4.213115]\n",
            "******* 174 [D loss: 0.009654, acc: 100.00%] [G loss: 4.186604]\n",
            "******* 175 [D loss: 0.013229, acc: 100.00%] [G loss: 4.282515]\n",
            "******* 176 [D loss: 0.009973, acc: 100.00%] [G loss: 4.315389]\n",
            "******* 177 [D loss: 0.010999, acc: 100.00%] [G loss: 4.242590]\n",
            "******* 178 [D loss: 0.012218, acc: 100.00%] [G loss: 4.266191]\n",
            "******* 179 [D loss: 0.012456, acc: 100.00%] [G loss: 4.224841]\n",
            "******* 180 [D loss: 0.010897, acc: 100.00%] [G loss: 4.221921]\n",
            "******* 181 [D loss: 0.012384, acc: 100.00%] [G loss: 4.153231]\n",
            "******* 182 [D loss: 0.011222, acc: 100.00%] [G loss: 4.293744]\n",
            "******* 183 [D loss: 0.007915, acc: 100.00%] [G loss: 4.245959]\n",
            "******* 184 [D loss: 0.011411, acc: 100.00%] [G loss: 4.280692]\n",
            "******* 185 [D loss: 0.012344, acc: 100.00%] [G loss: 4.282751]\n",
            "******* 186 [D loss: 0.011200, acc: 100.00%] [G loss: 4.272342]\n",
            "******* 187 [D loss: 0.013320, acc: 100.00%] [G loss: 4.303705]\n",
            "******* 188 [D loss: 0.009549, acc: 100.00%] [G loss: 4.353040]\n",
            "******* 189 [D loss: 0.009165, acc: 100.00%] [G loss: 4.358019]\n",
            "******* 190 [D loss: 0.012916, acc: 100.00%] [G loss: 4.264126]\n",
            "******* 191 [D loss: 0.010233, acc: 100.00%] [G loss: 4.249420]\n",
            "******* 192 [D loss: 0.011190, acc: 100.00%] [G loss: 4.318624]\n",
            "******* 193 [D loss: 0.008964, acc: 100.00%] [G loss: 4.312602]\n",
            "******* 194 [D loss: 0.010020, acc: 100.00%] [G loss: 4.398134]\n",
            "******* 195 [D loss: 0.010752, acc: 100.00%] [G loss: 4.277165]\n",
            "******* 196 [D loss: 0.011259, acc: 100.00%] [G loss: 4.253765]\n",
            "******* 197 [D loss: 0.010714, acc: 100.00%] [G loss: 4.291836]\n",
            "******* 198 [D loss: 0.009577, acc: 100.00%] [G loss: 4.337568]\n",
            "******* 199 [D loss: 0.010972, acc: 100.00%] [G loss: 4.321341]\n",
            "******* 200 [D loss: 0.011418, acc: 100.00%] [G loss: 4.362730]\n",
            "0.00000002\n",
            "saved\n",
            "******* 201 [D loss: 0.009933, acc: 100.00%] [G loss: 4.308037]\n",
            "******* 202 [D loss: 0.010024, acc: 100.00%] [G loss: 4.347883]\n",
            "******* 203 [D loss: 0.009041, acc: 100.00%] [G loss: 4.310122]\n",
            "******* 204 [D loss: 0.008538, acc: 100.00%] [G loss: 4.348898]\n",
            "******* 205 [D loss: 0.010756, acc: 100.00%] [G loss: 4.205606]\n",
            "******* 206 [D loss: 0.010113, acc: 100.00%] [G loss: 4.316840]\n",
            "******* 207 [D loss: 0.010625, acc: 100.00%] [G loss: 4.221187]\n",
            "******* 208 [D loss: 0.013296, acc: 100.00%] [G loss: 4.273458]\n",
            "******* 209 [D loss: 0.011574, acc: 100.00%] [G loss: 4.297751]\n",
            "******* 210 [D loss: 0.009893, acc: 100.00%] [G loss: 4.240689]\n",
            "******* 211 [D loss: 0.011143, acc: 100.00%] [G loss: 4.160333]\n",
            "******* 212 [D loss: 0.011955, acc: 100.00%] [G loss: 4.177412]\n",
            "******* 213 [D loss: 0.010818, acc: 100.00%] [G loss: 4.215970]\n",
            "******* 214 [D loss: 0.012480, acc: 100.00%] [G loss: 4.313668]\n",
            "******* 215 [D loss: 0.013620, acc: 100.00%] [G loss: 4.289402]\n",
            "******* 216 [D loss: 0.009655, acc: 100.00%] [G loss: 4.333922]\n",
            "******* 217 [D loss: 0.011663, acc: 100.00%] [G loss: 4.295591]\n",
            "******* 218 [D loss: 0.013272, acc: 100.00%] [G loss: 4.246381]\n",
            "******* 219 [D loss: 0.011881, acc: 100.00%] [G loss: 4.329163]\n",
            "******* 220 [D loss: 0.014982, acc: 100.00%] [G loss: 4.371473]\n",
            "******* 221 [D loss: 0.013422, acc: 100.00%] [G loss: 4.262572]\n",
            "******* 222 [D loss: 0.010737, acc: 100.00%] [G loss: 4.356739]\n",
            "******* 223 [D loss: 0.013434, acc: 100.00%] [G loss: 4.371290]\n",
            "******* 224 [D loss: 0.013243, acc: 100.00%] [G loss: 4.367368]\n",
            "******* 225 [D loss: 0.014152, acc: 100.00%] [G loss: 4.464168]\n",
            "******* 226 [D loss: 0.012541, acc: 100.00%] [G loss: 4.245787]\n",
            "******* 227 [D loss: 0.009646, acc: 100.00%] [G loss: 4.227634]\n",
            "******* 228 [D loss: 0.015385, acc: 100.00%] [G loss: 4.212928]\n",
            "******* 229 [D loss: 0.012583, acc: 100.00%] [G loss: 4.321209]\n",
            "******* 230 [D loss: 0.012570, acc: 100.00%] [G loss: 4.314577]\n",
            "******* 231 [D loss: 0.012439, acc: 100.00%] [G loss: 4.377110]\n",
            "******* 232 [D loss: 0.012377, acc: 100.00%] [G loss: 4.293907]\n",
            "******* 233 [D loss: 0.014652, acc: 100.00%] [G loss: 4.297708]\n",
            "******* 234 [D loss: 0.012556, acc: 100.00%] [G loss: 4.426029]\n",
            "******* 235 [D loss: 0.011466, acc: 100.00%] [G loss: 4.326212]\n"
          ]
        },
        {
          "output_type": "error",
          "ename": "KeyboardInterrupt",
          "evalue": "ignored",
          "traceback": [
            "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
            "\u001b[0;31mKeyboardInterrupt\u001b[0m                         Traceback (most recent call last)",
            "\u001b[0;32m<ipython-input-7-08179edae6ce>\u001b[0m in \u001b[0;36m<module>\u001b[0;34m()\u001b[0m\n\u001b[1;32m     39\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     40\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 41\u001b[0;31m \u001b[0mtrain\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;36m30000\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mbatch_size\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0;36m64\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0msave_interval\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0;36m200\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m",
            "\u001b[0;32m<ipython-input-7-08179edae6ce>\u001b[0m in \u001b[0;36mtrain\u001b[0;34m(epochs, batch_size, save_interval)\u001b[0m\n\u001b[1;32m     19\u001b[0m     \u001b[0;31m#Generate Fake Images\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     20\u001b[0m     \u001b[0mnoise\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mnp\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrandom\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mnormal\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;36m0\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;36m1\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m(\u001b[0m\u001b[0mbatch_size\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mlatent_dim\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 21\u001b[0;31m     \u001b[0mgen_imgs\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mgenerator\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mpredict\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mnoise\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     22\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     23\u001b[0m     \u001b[0;31m#Train discriminator\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py\u001b[0m in \u001b[0;36merror_handler\u001b[0;34m(*args, **kwargs)\u001b[0m\n\u001b[1;32m     62\u001b[0m     \u001b[0mfiltered_tb\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     63\u001b[0m     \u001b[0;32mtry\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 64\u001b[0;31m       \u001b[0;32mreturn\u001b[0m \u001b[0mfn\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     65\u001b[0m     \u001b[0;32mexcept\u001b[0m \u001b[0mException\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0me\u001b[0m\u001b[0;34m:\u001b[0m  \u001b[0;31m# pylint: disable=broad-except\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     66\u001b[0m       \u001b[0mfiltered_tb\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0m_process_traceback_frames\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0me\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m__traceback__\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/keras/engine/training.py\u001b[0m in \u001b[0;36mpredict\u001b[0;34m(self, x, batch_size, verbose, steps, callbacks, max_queue_size, workers, use_multiprocessing)\u001b[0m\n\u001b[1;32m   1766\u001b[0m           \u001b[0muse_multiprocessing\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0muse_multiprocessing\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1767\u001b[0m           \u001b[0mmodel\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 1768\u001b[0;31m           steps_per_execution=self._steps_per_execution)\n\u001b[0m\u001b[1;32m   1769\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1770\u001b[0m       \u001b[0;31m# Container that configures and calls `tf.keras.Callback`s.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/keras/engine/data_adapter.py\u001b[0m in \u001b[0;36mget_data_handler\u001b[0;34m(*args, **kwargs)\u001b[0m\n\u001b[1;32m   1401\u001b[0m   \u001b[0;32mif\u001b[0m \u001b[0mgetattr\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0;34m\"model\"\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m\"_cluster_coordinator\"\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1402\u001b[0m     \u001b[0;32mreturn\u001b[0m \u001b[0m_ClusterCoordinatorDataHandler\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 1403\u001b[0;31m   \u001b[0;32mreturn\u001b[0m \u001b[0mDataHandler\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   1404\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1405\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/keras/engine/data_adapter.py\u001b[0m in \u001b[0;36m__init__\u001b[0;34m(self, x, y, sample_weight, batch_size, steps_per_epoch, initial_epoch, epochs, shuffle, class_weight, max_queue_size, workers, use_multiprocessing, model, steps_per_execution, distribute)\u001b[0m\n\u001b[1;32m   1163\u001b[0m         \u001b[0muse_multiprocessing\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0muse_multiprocessing\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1164\u001b[0m         \u001b[0mdistribution_strategy\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mtf\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdistribute\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mget_strategy\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 1165\u001b[0;31m         model=model)\n\u001b[0m\u001b[1;32m   1166\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1167\u001b[0m     \u001b[0mstrategy\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mtf\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdistribute\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mget_strategy\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/keras/engine/data_adapter.py\u001b[0m in \u001b[0;36m__init__\u001b[0;34m(self, x, y, sample_weights, sample_weight_modes, batch_size, epochs, steps, shuffle, **kwargs)\u001b[0m\n\u001b[1;32m    289\u001b[0m     \u001b[0;31m# trigger the next permutation. On the other hand, too many simultaneous\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    290\u001b[0m     \u001b[0;31m# shuffles can contend on a hardware level and degrade all performance.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 291\u001b[0;31m     \u001b[0mindices_dataset\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mindices_dataset\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mmap\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mpermutation\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mprefetch\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;36m1\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    292\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    293\u001b[0m     \u001b[0;32mdef\u001b[0m \u001b[0mslice_batch_indices\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mindices\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py\u001b[0m in \u001b[0;36mmap\u001b[0;34m(self, map_func, num_parallel_calls, deterministic, name)\u001b[0m\n\u001b[1;32m   2002\u001b[0m         warnings.warn(\"The `deterministic` argument has no effect unless the \"\n\u001b[1;32m   2003\u001b[0m                       \"`num_parallel_calls` argument is specified.\")\n\u001b[0;32m-> 2004\u001b[0;31m       \u001b[0;32mreturn\u001b[0m \u001b[0mMapDataset\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mmap_func\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mpreserve_cardinality\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0;32mTrue\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mname\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mname\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   2005\u001b[0m     \u001b[0;32melse\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   2006\u001b[0m       return ParallelMapDataset(\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py\u001b[0m in \u001b[0;36m__init__\u001b[0;34m(self, input_dataset, map_func, use_inter_op_parallelism, preserve_cardinality, use_legacy_function, name)\u001b[0m\n\u001b[1;32m   5457\u001b[0m         \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_transformation_name\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   5458\u001b[0m         \u001b[0mdataset\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0minput_dataset\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 5459\u001b[0;31m         use_legacy_function=use_legacy_function)\n\u001b[0m\u001b[1;32m   5460\u001b[0m     \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_metadata\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mdataset_metadata_pb2\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mMetadata\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   5461\u001b[0m     \u001b[0;32mif\u001b[0m \u001b[0mname\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py\u001b[0m in \u001b[0;36m__init__\u001b[0;34m(self, func, transformation_name, dataset, input_classes, input_shapes, input_types, input_structure, add_to_graph, use_legacy_function, defun_kwargs)\u001b[0m\n\u001b[1;32m   4531\u001b[0m         \u001b[0mfn_factory\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mtrace_tf_function\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mdefun_kwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   4532\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 4533\u001b[0;31m     \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_function\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mfn_factory\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   4534\u001b[0m     \u001b[0;31m# There is no graph to add in eager mode.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   4535\u001b[0m     \u001b[0madd_to_graph\u001b[0m \u001b[0;34m&=\u001b[0m \u001b[0;32mnot\u001b[0m \u001b[0mcontext\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mexecuting_eagerly\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/function.py\u001b[0m in \u001b[0;36mget_concrete_function\u001b[0;34m(self, *args, **kwargs)\u001b[0m\n\u001b[1;32m   3243\u001b[0m     \"\"\"\n\u001b[1;32m   3244\u001b[0m     graph_function = self._get_concrete_function_garbage_collected(\n\u001b[0;32m-> 3245\u001b[0;31m         *args, **kwargs)\n\u001b[0m\u001b[1;32m   3246\u001b[0m     \u001b[0mgraph_function\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_garbage_collector\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrelease\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m  \u001b[0;31m# pylint: disable=protected-access\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   3247\u001b[0m     \u001b[0;32mreturn\u001b[0m \u001b[0mgraph_function\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/function.py\u001b[0m in \u001b[0;36m_get_concrete_function_garbage_collected\u001b[0;34m(self, *args, **kwargs)\u001b[0m\n\u001b[1;32m   3208\u001b[0m       \u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mkwargs\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   3209\u001b[0m     \u001b[0;32mwith\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_lock\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 3210\u001b[0;31m       \u001b[0mgraph_function\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0m_\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_maybe_define_function\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   3211\u001b[0m       \u001b[0mseen_names\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mset\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   3212\u001b[0m       captured = object_identity.ObjectIdentitySet(\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/function.py\u001b[0m in \u001b[0;36m_maybe_define_function\u001b[0;34m(self, args, kwargs)\u001b[0m\n\u001b[1;32m   3555\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   3556\u001b[0m           \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_function_cache\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mmissed\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0madd\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mcall_context_key\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 3557\u001b[0;31m           \u001b[0mgraph_function\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_create_graph_function\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   3558\u001b[0m           \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_function_cache\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mprimary\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0mcache_key\u001b[0m\u001b[0;34m]\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mgraph_function\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   3559\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/function.py\u001b[0m in \u001b[0;36m_create_graph_function\u001b[0;34m(self, args, kwargs, override_flat_arg_shapes)\u001b[0m\n\u001b[1;32m   3400\u001b[0m             \u001b[0marg_names\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0marg_names\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   3401\u001b[0m             \u001b[0moverride_flat_arg_shapes\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0moverride_flat_arg_shapes\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 3402\u001b[0;31m             capture_by_value=self._capture_by_value),\n\u001b[0m\u001b[1;32m   3403\u001b[0m         \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_function_attributes\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   3404\u001b[0m         \u001b[0mfunction_spec\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mfunction_spec\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/func_graph.py\u001b[0m in \u001b[0;36mfunc_graph_from_py_func\u001b[0;34m(name, python_func, args, kwargs, signature, func_graph, autograph, autograph_options, add_control_dependencies, arg_names, op_return_value, collections, capture_by_value, override_flat_arg_shapes, acd_record_initial_resource_uses)\u001b[0m\n\u001b[1;32m   1026\u001b[0m   \u001b[0;32mif\u001b[0m \u001b[0mfunc_graph\u001b[0m \u001b[0;32mis\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1027\u001b[0m     func_graph = FuncGraph(name, collections=collections,\n\u001b[0;32m-> 1028\u001b[0;31m                            capture_by_value=capture_by_value)\n\u001b[0m\u001b[1;32m   1029\u001b[0m   \u001b[0;32massert\u001b[0m \u001b[0misinstance\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mfunc_graph\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mFuncGraph\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1030\u001b[0m   \u001b[0;32mif\u001b[0m \u001b[0madd_control_dependencies\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/func_graph.py\u001b[0m in \u001b[0;36m__init__\u001b[0;34m(self, name, collections, capture_by_value)\u001b[0m\n\u001b[1;32m    188\u001b[0m         \u001b[0;32mfrom\u001b[0m \u001b[0mouter\u001b[0m \u001b[0mgraphs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;32mand\u001b[0m \u001b[0mfailing\u001b[0m \u001b[0mthat\u001b[0m \u001b[0mwill\u001b[0m \u001b[0mdefault\u001b[0m \u001b[0mto\u001b[0m \u001b[0;32mFalse\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    189\u001b[0m     \"\"\"\n\u001b[0;32m--> 190\u001b[0;31m     \u001b[0msuper\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mFuncGraph\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m__init__\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    191\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    192\u001b[0m     \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mname\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mname\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/ops.py\u001b[0m in \u001b[0;36m__init__\u001b[0;34m(self)\u001b[0m\n\u001b[1;32m   3020\u001b[0m     \u001b[0;31m# Similarly, if one or more Session.run calls are going on, all mutate ops\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   3021\u001b[0m     \u001b[0;31m# have to wait until all Session.run calls have finished.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 3022\u001b[0;31m     \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_group_lock\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mlock_util\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mGroupLock\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mnum_groups\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0;36m2\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   3023\u001b[0m     \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_nodes_by_id\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;34m{\u001b[0m\u001b[0;34m}\u001b[0m  \u001b[0;31m# GUARDED_BY(self._lock)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   3024\u001b[0m     \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_next_id_counter\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;36m0\u001b[0m  \u001b[0;31m# GUARDED_BY(self._lock)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/lock_util.py\u001b[0m in \u001b[0;36m__init__\u001b[0;34m(self, num_groups)\u001b[0m\n\u001b[1;32m     71\u001b[0m           \u001b[0;34m\"Argument `num_groups` must be a positive integer. \"\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     72\u001b[0m           f\"Received: num_groups={num_groups}\")\n\u001b[0;32m---> 73\u001b[0;31m     \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_ready\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mthreading\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mCondition\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mthreading\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mLock\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     74\u001b[0m     \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_num_groups\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mnum_groups\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     75\u001b[0m     \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_group_member_counts\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;34m[\u001b[0m\u001b[0;36m0\u001b[0m\u001b[0;34m]\u001b[0m \u001b[0;34m*\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_num_groups\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/lib/python3.7/threading.py\u001b[0m in \u001b[0;36m__init__\u001b[0;34m(self, lock)\u001b[0m\n\u001b[1;32m    220\u001b[0m         \u001b[0;31m# Export the lock's acquire() and release() methods\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    221\u001b[0m         \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0macquire\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mlock\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0macquire\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 222\u001b[0;31m         \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrelease\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mlock\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrelease\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    223\u001b[0m         \u001b[0;31m# If the lock defines _release_save() and/or _acquire_restore(),\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    224\u001b[0m         \u001b[0;31m# these override the default implementations (which just call\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;31mKeyboardInterrupt\u001b[0m: "
          ]
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "po-jSQoN1Azl"
      },
      "source": [
        "### **8) Making GIF**"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "XPShgQpg1EMy"
      },
      "source": [
        "# Display a single image using the epoch number\n",
        "# def display_image(epoch_no):\n",
        "#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))\n",
        "\n",
        "anim_file = 'dcgan.gif'\n",
        "\n",
        "with imageio.get_writer(anim_file, mode='I') as writer:\n",
        "  filenames = glob.glob('generated_images/*.png')\n",
        "  filenames = sorted(filenames)\n",
        "  for filename in filenames:\n",
        "    image = imageio.imread(filename)\n",
        "    writer.append_data(image)\n",
        "  image = imageio.imread(filename)\n",
        "  writer.append_data(image)"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "Wh37uv1torG5"
      },
      "source": [
        ""
      ],
      "execution_count": null,
      "outputs": []
    }
  ]
}